# Extracting per-read per-position dwell-times from fast5 files (notebook 2)

The last notebook was getting too long. We located the data, let's try to extract it in this notebook.

In [15]:
import numpy as np
import numpy.lib.recfunctions as rfn # tools for working with structured arrays
import h5py # standard Python library for working with HDF5 files - http://docs.h5py.org/en/stable/quick.html

In [2]:
fast5_path = 'data/01936724-8ae4-4a0f-99ce-c67f3a334c1f.fast5'
reference_fasta_path = 'data/RNA_section__454_9627.fa'

In [3]:
# Chris Viola wrote this nice code to recursively describe the whole HDF5 file structure

def explore_hdf5_file(hdf5_group,tab_str):
    '''
    Given an HDF5 root key, hdf5_group prints info on its name, attributes, and subgroups. Does so 
    for the entire tree recursively. 
    '''
    assert isinstance(hdf5_group, h5py._hl.group.Group), 'Error: This is not an hdf5 group. Its type is {}'.format(type(hdf5_group))
    
    print(tab_str + 'group: {}'.format(hdf5_group))
    
    print(tab_str + 'attribute values:')
    for key in hdf5_group.attrs.keys():
        print(tab_str + 'attribute: {}'.format(key)) #attr name 
        print(tab_str + 'value: {}'.format(hdf5_group.attrs[key])) #value of attr
    
    datasets_and_subgroups = hdf5_group.keys()
    datasets = [x for x in datasets_and_subgroups if isinstance(hdf5_group[x], h5py._hl.dataset.Dataset)]
    subgroups = [x for x in datasets_and_subgroups if isinstance(hdf5_group[x], h5py._hl.group.Group)]
    print(tab_str + 'datasets: {}'.format(datasets))
    print(tab_str + 'subgroups: {}'.format(subgroups))
    print('')
    tab_str+='\t'
    if len(subgroups) != 0:
        for subgroup in subgroups:
            explore_hdf5_file(hdf5_group[subgroup],tab_str)
            
with h5py.File(fast5_path, 'r') as file:
    explore_hdf5_file(file,'')

group: <HDF5 file "01936724-8ae4-4a0f-99ce-c67f3a334c1f.fast5" (mode r)>
attribute values:
attribute: file_version
value: 2.0
datasets: []
subgroups: [u'Analyses', u'Raw', u'UniqueGlobalKey']

	group: <HDF5 group "/Analyses" (3 members)>
	attribute values:
	datasets: []
	subgroups: [u'Segmentation_000', u'Basecall_1D_000', u'RawGenomeCorrected_000']

		group: <HDF5 group "/Analyses/Segmentation_000" (1 members)>
		attribute values:
		attribute: name
		value: MinKNOW-Live-Basecalling
		attribute: version
		value: 3.6.0
		attribute: time_stamp
		value: 2020-01-08T12:27:08Z
		datasets: []
		subgroups: [u'Summary']

			group: <HDF5 group "/Analyses/Segmentation_000/Summary" (1 members)>
			attribute values:
			attribute: return_status
			value: Workflow successful
			datasets: []
			subgroups: [u'segmentation']

				group: <HDF5 group "/Analyses/Segmentation_000/Summary/segmentation" (0 members)>
				attribute values:
				attribute: duration_template
				value: 409005
				attribute: first_

One insight from looking at all the attributes in the file:
it appears attribute `sample_frequency` of `/UniqueGlobalKey/context_tags` tells us the sampling frequency in Hz.
(I have not confirmed this!!)

In [4]:
# store sample frequency for later use
with h5py.File(fast5_path, 'r') as file:
    sample_frequency = file['/UniqueGlobalKey/context_tags'].attrs['sample_frequency']

In [5]:
# look at Events dataset
with h5py.File(fast5_path, 'r') as file:
    events_dset = file['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Events']
    print(type(events_dset))
    print(events_dset)
    print(events_dset.dtype)
    print(events_dset[()].reshape(-1,1)) # reshape function is just to make rows line up in a column

<class 'h5py._hl.dataset.Dataset'>
<HDF5 dataset "Events": shape (9133,), type "|V25">
[('norm_mean', '<f8'), ('norm_stdev', '<f8'), ('start', '<u4'), ('length', '<u4'), ('base', 'S1')]
[[(2.1117437 , nan,      0, 20, 'G')]
 [(2.31800499, nan,     20, 17, 'A')]
 [(1.93237796, nan,     37,  6, 'G')]
 ...
 [(1.66882629, nan, 405536, 27, 'G')]
 [(2.28478419, nan, 405563, 12, 'A')]
 [(0.71333169, nan, 405575, 11, 'G')]]


In [6]:
# convert Events dataset to numpy array
with h5py.File(fast5_path, 'r') as file:
    events_dset = file['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Events']
    events_array = np.asarray(events_dset)
    
print(events_array.dtype)
print(events_array.reshape(-1,1)) # reshape function is just to make rows line up in a column

[('norm_mean', '<f8'), ('norm_stdev', '<f8'), ('start', '<u4'), ('length', '<u4'), ('base', 'S1')]
[[(2.1117437 , nan,      0, 20, 'G')]
 [(2.31800499, nan,     20, 17, 'A')]
 [(1.93237796, nan,     37,  6, 'G')]
 ...
 [(1.66882629, nan, 405536, 27, 'G')]
 [(2.28478419, nan, 405563, 12, 'A')]
 [(0.71333169, nan, 405575, 11, 'G')]]


In [7]:
# double-check that the genetic sequence in events_array is a subsequence of the reference genome

# Here's the sequence from the events table
events_sequence = ''.join(events_array['base']) # concatenate letters from "length" column of table

# Here's the reference genome
with open(reference_fasta_path, 'rt') as file:
    # For some reason, file.read() outputs a string including "\r\n". That's supposed to be a newline separator
    reference_genome = file.read().split('\r\n')[1] # split by the weird newline separator, then access the second line with "[1]"

# This method would need modification to work on more sophisticated reference genomes with
# multiple isoforms, multiple chromosomes, multiple strands, etc.
assert events_sequence.upper() in reference_genome.upper() # one of these strings is lowercase, the other upper, so we must convert
print('The sequence from `events_array` is indeed a subsequence of the reference genome')

The sequence from `events_array` is indeed a subsequence of the reference genome


Let's just create an array called `pos_array` (of the same shape as `events_array`) that describes genomic position.
For each entry in `events_array`, the corresponding entry in `pos_array` will be
the position of the corresponding nucleotide in the reference genome.
We will use zero-based indexing into the reference genome, which is (hopefully!) compatible with Tombo.

HDF5 group `/Analyses/RawGenomeCorrected_000/BaseCalled_template/Alignment/` attribute `mapped_start` gives critical information for this.
It is the offset between (zero-based) row-index in the Events table and (zero-based) character-index in the reference genome sequence string.
(The attribute `mapped_end` gives basically the same info, but I think it's redundant given the length of `events_array`. See the assertion in the following code block.)

In [8]:
# get parameters to align `events_array` with reference genome
with h5py.File(fast5_path, 'r') as file:
    mapped_start = file['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Alignment/'].attrs['mapped_start']
    mapped_end = file['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Alignment/'].attrs['mapped_end']

assert mapped_start + len(events_array) == mapped_end

The following code block contains an important assertion. It makes sure we are indexing correctly into the reference genome.

In [49]:
# create index based on genomic nucleotide position to go with events_table
ref_genome_pos = np.arange(mapped_start, mapped_start + len(events_array))

# check that for all i the following sentence is true:
#     "The base in row i of events_array equals the base in row ref_genome_pos[i] of reference_genome."
assert all([reference_genome[ref_genome_pos[i]].upper() == base.upper()
            for i, base in enumerate(events_array['base'])])

In [58]:
# take the index based on reference-genome nucleotide position, and the 'lengths' column, and combine them into one array

assert events_array.shape == ref_genome_pos.shape

ref_pos_and_length = ( # name means "reference genome position and length, side by side in the same array
    rfn.merge_arrays( # create structured array with two columns...
        [
            ref_genome_pos, # ...one column from `ref_genome_pos`
            events_array['length'] # ...the other column from the `'length'` field of `events_array`
        ],
        flatten=True
    )
    .view( # name columns appropriately
        np.dtype(
            [
                 ('reference_genome_position', '<i8'),
                 ('length', '<u4')
            ]
        )
    )
)

# Check again that the reference genome position is correct
assert np.array_equal([reference_genome[pos].upper() for pos in ref_pos_and_length['reference_genome_position']], events_array['base'])

ref_pos_and_length.reshape(-1,1) # reshape just to make it display in one column

array([[(  33, 20)],
       [(  34, 17)],
       [(  35,  6)],
       ...,
       [(9163, 27)],
       [(9164, 12)],
       [(9165, 11)]],
      dtype=[('reference_genome_position', '<i8'), ('length', '<u4')])

In [77]:
# dump ref_pos_and_length to CSV
csv_header = 'reference_genome_position,length\n'
csv_body = ''.join(['{},{}\n'.format(*row) for row in ref_pos_and_length])
csv_text = csv_header + csv_body
with open('positions_and_lengths.csv', 'wt') as f:
    f.write(csv_text)

## The End?

We succeeded at dumping single-read dwell times to a CSV file, indexed by position on the reference genome. Next steps:
* Make a script that can do this automatically
* Give dwell times in milliseconds, rather than in multiples of the sampling frequency